In [ ]:
# !pip install transformers
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install transformers
!pip install demoji
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 3.3 MB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 61 kB 461 kB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 42 kB 902 kB/s 
     |████████████████████████████████| 1.3 MB 9.8 MB/s 
     |████████████████████████████████| 65 kB 3.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=d0d58522a779e55c056c95ced4db0103547d628faa7f06e7f1387f0b2aa27478
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ff

In [ ]:
import pandas as pd
print(pd.__version__)
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
import unicodedata
from googletrans import Translator
import demoji

1.1.5


In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Load Twitter Dataset Files:  Training & Test 

In [ ]:
twitter_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/trainingandtestdata/training.1600000.processed.noemoticon.csv',encoding='latin1',header=None)
twitter_test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/trainingandtestdata/testdata.manual.2009.06.14.csv',encoding='latin1',header=None)

## Preprocess the Training and Test data

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join( c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def trans_text(df, col):
    translator = Translator()
    translations = {}
    # translations[t] = translator.translate(t,dest='en').text
    cnt = 0
    df_en = df.copy()
    translations = {}
    # unique elements of the column
    unique_elements = df_en[col].unique()
    for element in unique_elements:
        # add translation to the dictionary
        detected_eq = translator.detect(element)
        if detected_eq.lang!='en' and detected_eq.confidence> 0.5:
            print(element)
            translations[element] = translator.translate(element).text
            cnt+=1
    return translations, int(cnt)
    
print(demoji.replace_with_desc('▪', " "))
translator = Translator()
try:
    trans = translator.translate('안녕하세요.',dest='en')
    translation = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')
except BaseException as err:
    print('Error returned')
    print(f"Unexpected {err} =, {type(err)}")
print(translation.text)
print(trans.text)

 black small square 
The sky is blue and I like bananas
Hello.


In [ ]:
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"#[\w]+":""}

def preprocess(datainput):
    t=datainput
    t = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', t, flags=re.MULTILINE)
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def remove_emoji(datainput):
    t=datainput
    t = demoji.replace_with_desc(t, " ")
    return t

def no_bracket(datainput):
    t=datainput
    t = t[2:-2]
    return t


def lower_text(datainput):
    t=datainput
    t = unicodeToAscii(t.lower().strip())
    # Lowercase, trim, and remove non-letter characters
    t = re.sub(r"([.!?])", r" \1", t)
    t = re.sub(r"[^a-zA-Z.!?]+", r" ", t)
    return t

In [ ]:
# Prep the training data: select the subset of data fields, name the columns, remove usernames and URL links
twitter_train=twitter_train[[0,5]]
twitter_train.columns=["sentiment", "tweet"]
twitter_train['tweet']=twitter_train['tweet'].apply(preprocess)
twitter_train.head()

,sentiment,tweet
0,0,"Awww, thats a bummer. You shoulda got Davi..."
1,0,is upset that he cant update his Facebook by t...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, its not behaving at all. im mad. why am i..."


In [ ]:

# TRANSLATE TO ENGLISH
df = trans_text(twitter_train,'tweet')
twitter_train['tweet'] = df['tweet']
print(df.head())
print(twitter_train.head())

In [ ]:
# /// twitter_train.to_csv('./Translated_Training.csv', header=None, index=False,encoding='latin1')
twitter_train['tweet']=twitter_train['tweet'].apply(remove_emoji)
twitter_train.head()

In [ ]:
# twitter_train.to_csv('./Noemoji_Training.csv',header=None, index=False,encoding='latin1')
twitter_train['tweet']=twitter_train['tweet'].apply(lower_text)
twitter_train.to_csv('/content/drive/My Drive/Colab Notebooks/trainingandtestdata/Lower_Training.csv', header=None, index=False,encoding='latin1')
twitter_train.head()

In [ ]:
# Prep the test data

twitter_test=twitter_test.loc[:,(0,5)]
twitter_test.columns=["sentiment", "tweet"]
twitter_test["tweet"]=twitter_test["tweet"].apply(preprocess)
twitter_test.head()

In [ ]:
twitter_test['tweet']=twitter_test['tweet'].apply(remove_emoji)
twitter_test.head()
twitter_test['tweet']=twitter_test['tweet'].apply(lower_text)
twitter_test.to_csv('/content/drive/My Drive/Colab Notebooks/trainingandtestdata/Lower_Test.csv', header=None, index=False,encoding='latin1')
twitter_test.head()

In [ ]:
print(len(twitter_train))
print(len(twitter_test))

In [ ]:
# #Switch out half of the no-emotion dataset in the test file for other data in the train file
# a = twitter_train[(twitter_train['sentiment']==0)][0:34]
# b = twitter_train[(twitter_train['sentiment']==4)][60:95]
# c = twitter_test[(twitter_test['sentiment']==2)][0:34]
# d = twitter_test[(twitter_test['sentiment']==2)][60:95]
# ai = twitter_train.index[(twitter_train['sentiment']==0)][0:34]
# bi = twitter_train.index[(twitter_train['sentiment']==4)][60:95]
# ci = twitter_test.index[(twitter_test['sentiment']==2)][0:34]
# di = twitter_test.index[(twitter_test['sentiment']==2)][60:95]
# twitter_train = twitter_train.drop(ai)
# twitter_train = twitter_train.drop(bi)
# twitter_train = twitter_train.append(c)
# twitter_train = twitter_train.append(d)
# twitter_test = twitter_test.drop(ci)
# twitter_test = twitter_test.drop(di)
# twitter_test = twitter_test.append(a)
# twitter_test = twitter_test.append(b)

# Try dropping the neutral sentiments from test per instructor's notes
ai = twitter_test.index[(twitter_test['sentiment']==2)]
twitter_test = twitter_test.drop(ai)

print(len(twitter_train))
print(len(twitter_test))

print(twitter_train['tweet'][6800])
# print(twitter_test['tweet'][200])

In [ ]:
vectorizer = CountVectorizer()

# Fit Count Vectorizer on training data

In [ ]:
vectorizer.fit(twitter_train['tweet'].values)

In [ ]:
vectorizer.vocabulary_

## Stopwords

In [ ]:
stop_words_vectorizer=CountVectorizer(stop_words='english')
stop_words_vectorizer.fit(twitter_train['tweet'].values)


In [ ]:
stop_words_vectorizer.vocabulary_

In [ ]:
x_input=stop_words_vectorizer.transform(twitter_train['tweet'].values)
x_test_input=stop_words_vectorizer.transform(twitter_test['tweet'].values)
y_input=twitter_train['sentiment'].values
y_test=twitter_test['sentiment'].values

In [ ]:
nb = MultinomialNB(alpha=1.0)

In [ ]:
nb.fit(x_input,y_input)

In [ ]:
y_pred_class = nb.predict(x_test_input)
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
print(classification_report(y_true=y_test,y_pred=y_pred_class))

In [ ]:
len(twitter_train[(twitter_train['sentiment']==0)])

In [ ]:
len(twitter_train[(twitter_train['sentiment']==2)])

In [ ]:
len(twitter_train[(twitter_train['sentiment']==4)])

In [ ]:
len(pd.unique(twitter_train['sentiment']))


In [ ]:
len(pd.unique(twitter_test['sentiment']))

In [ ]:
twitter_train.sentiment.unique()

In [ ]:
twitter_test.sentiment.unique()

In [ ]:
len(twitter_test[(twitter_test['sentiment']==0)])

In [ ]:
len(twitter_test[(twitter_test['sentiment']==2)])

In [ ]:
len(twitter_test[(twitter_test['sentiment']==4)])

In [ ]:
b=twitter_test.index[(twitter_test['sentiment']==2)]
b[0:34]
b[60:95]

In [ ]:
a=twitter_test.index[(twitter_test['sentiment']==0)]
a[0:34]

In [ ]:
c=twitter_test.index[(twitter_test['sentiment']==4)]
c[60:95]